In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import s3fs

import matplotlib.pyplot as plt
import seaborn as sns

import os
from datetime import datetime

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [3]:
# data ingestion
df = pd.read_parquet('../data/Bronze/HI-Medium_Trans.parquet') # S3_BUCKET = 's3://bass-risk-monitoring/Bronze/HI-Medium_Trans.csv'

print(f"Dataframe was loaded successfully")


Dataframe was loaded successfully


In [4]:

print(f"Shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**3:.2f} GB")

Shape: (31898238, 11)
Memory usage: 13.02 GB


Initial Data Inspection

In [5]:
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:17,20,800104D70,20,800104D70,6794.63,US Dollar,6794.63,US Dollar,Reinvestment,0
1,2022/09/01 00:02,3196,800107150,3196,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0
2,2022/09/01 00:17,1208,80010E430,1208,80010E430,1880.23,US Dollar,1880.23,US Dollar,Reinvestment,0
3,2022/09/01 00:03,1208,80010E650,20,80010E6F0,73966883.00,US Dollar,73966883.00,US Dollar,Cheque,0
4,2022/09/01 00:02,1208,80010E650,20,80010EA30,45868454.00,US Dollar,45868454.00,US Dollar,Cheque,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31898238 entries, 0 to 31898237
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           object 
 1   From Bank           int64  
 2   Account             object 
 3   To Bank             int64  
 4   Account.1           object 
 5   Amount Received     float64
 6   Receiving Currency  object 
 7   Amount Paid         float64
 8   Payment Currency    object 
 9   Payment Format      object 
 10  Is Laundering       int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 2.6+ GB


Data Quality

In [8]:
# missing values
print("Missing Values:")
missing = df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print(" No missing values found")

Missing Values:
 No missing values found


In [9]:

# duplicates
print("Duplicate Values:")
duplicates = df.duplicated().sum()

if duplicates > 0:
    print("Found",duplicates,"duplicate records")
    duplicate_records = df[df.duplicated(keep=False)].sort_values(by=list(df.columns))
else:
    print(" No dupliction reconds found")

Duplicate Values:
Found 20 duplicate records


In [10]:
duplicate_records

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
8159369,2022/09/03 18:39,122407,80E54B721,213589,84EC258A1,0.000002,Bitcoin,0.000002,Bitcoin,Bitcoin,0
8159371,2022/09/03 18:39,122407,80E54B721,213589,84EC258A1,0.000002,Bitcoin,0.000002,Bitcoin,Bitcoin,0
9834991,2022/09/05 07:36,161430,828C83141,161430,828C83140,0.000001,Bitcoin,0.010000,US Dollar,ACH,0
9834993,2022/09/05 07:36,161430,828C83141,161430,828C83140,0.000001,Bitcoin,0.010000,US Dollar,ACH,0
9834992,2022/09/05 07:36,161430,828C83141,2213875,851BB5C81,0.000001,Bitcoin,0.000001,Bitcoin,Bitcoin,0
9834994,2022/09/05 07:36,161430,828C83141,2213875,851BB5C81,0.000001,Bitcoin,0.000001,Bitcoin,Bitcoin,0
13710705,2022/09/07 08:15,2597,800AB57E0,2597,800AB57E0,0.050000,Euro,0.060000,US Dollar,ACH,0
13710707,2022/09/07 08:15,2597,800AB57E0,2597,800AB57E0,0.050000,Euro,0.060000,US Dollar,ACH,0
15819700,2022/09/08 10:14,1313,817576F91,1313,817576F90,0.000001,Bitcoin,0.010000,US Dollar,ACH,0
15819702,2022/09/08 10:14,1313,817576F91,1313,817576F90,0.000001,Bitcoin,0.010000,US Dollar,ACH,0


In [12]:
#  class distribution
class_distribution = df['Is Laundering'].value_counts()
class_distribution

Is Laundering
0    31863008
1       35230
Name: count, dtype: int64

In [13]:
(df['Is Laundering'].value_counts(normalize=True) * 100).round(2)

Is Laundering
0    99.89
1     0.11
Name: proportion, dtype: float64

In [15]:
# remove duplicates
df = df.drop_duplicates(keep="first").reset_index(drop=True)
print(f" duplicates removed")

 duplicates removed


Data Type conversion  to reduce memory usage

In [16]:
df.dtypes

Timestamp              object
From Bank               int64
Account                object
To Bank                 int64
Account.1              object
Amount Received       float64
Receiving Currency     object
Amount Paid           float64
Payment Currency       object
Payment Format         object
Is Laundering           int64
dtype: object

In [17]:

int_cols = ['From Bank', 'To Bank', 'Is Laundering']
for col in int_cols:
    df[col] = df[col].astype('int32')

float_cols = ['Amount Received', 'Amount Paid']
for col in float_cols:
    df[col] = df[col].astype('float32')

cat_cols = ['Receiving Currency', 'Payment Currency', 'Payment Format']
for col in cat_cols:
    df[col] = df[col].astype('category')


df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M')


memory_after = df.memory_usage(deep=True).sum() / 1024**3
print(f"Memory after: {memory_after:.2f} GB")
print(f"Reduction: {(((13.02 - memory_after) / 13.02) * 100).round(2)}%")


Memory after: 4.84 GB
Reduction: 62.81%


Descriptive statistics

In [19]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Timestamp,31898218,2022-09-08 15:00:42.711086336,2022-09-01 00:00:00,2022-09-03 13:22:00,2022-09-08 12:14:00,2022-09-13 10:06:00,2022-09-28 15:58:00,NaN
From Bank,31898218.0,294409.393878,0.0,2954.0,39024.0,215884.0,3225455.0,615314.827862
To Bank,31898218.0,409319.414736,0.0,27496.0,146853.0,259893.0,3225455.0,654699.85721
Amount Received,31898218.0,6431110.0,0.000001,207.869995,1469.26001,11835.299805,8158609145856.0,2592744960.0
Amount Paid,31898218.0,4417549.5,0.000001,209.229996,1471.540039,11757.80957,8158609145856.0,1848313984.0
Is Laundering,31898218.0,0.001104,0.0,0.0,0.0,0.0,1.0,0.033215


In [20]:
# categorical columns
for col in ['Receiving Currency', 'Payment Currency', 'Payment Format']:
    if col in df.columns:
        print(f"\n{col}:")
        print(df[col].value_counts().head(10))


Receiving Currency:
Receiving Currency
US Dollar            11594241
Euro                  7329169
Yuan                  2295849
Shekel                1428622
Canadian Dollar       1089398
UK Pound              1020042
Ruble                  989319
Australian Dollar      931364
Yen                    860154
Swiss Franc            856680
Name: count, dtype: int64

Payment Currency:
Payment Currency
US Dollar            11688243
Euro                  7315957
Yuan                  2330638
Shekel                1408707
Canadian Dollar       1078358
UK Pound              1018575
Ruble                  975525
Australian Dollar      918569
Yen                    856132
Swiss Franc            844623
Name: count, dtype: int64

Payment Format:
Payment Format
Cheque          12280058
Credit Card      8777816
ACH              3868404
Cash             3217531
Reinvestment     1945611
Wire             1119773
Bitcoin           689025
Name: count, dtype: int64


- US Dollar dominance in transactions (11.6M/31.9M)
- Euro second (7.3M transactions)
- Top 3 currencies (USD, EUR, CNY) account for all transactions
- 10 major currencies cover nearly all transactions
- Receiving vs Payment Currency: Nearly identical distributions (correlation suggests same-currency transactions dominate)

Save Cleaned Data to Bronze Layer

In [22]:

df.to_parquet('../data/Bronze/HI_Medium_cleaned.parquet', index=False, compression='snappy')
print("Clearned and Optimized data was saved to ../data/Bronze/HI_Medium_cleaned.parquet")

Clearned and Optimized data was saved to ../data/Bronze/HI_Medium_cleaned.parquet



**Achievements**:
-  Loaded 32M transactions 
-  Optimized memory usage 13.02 GB -> 4.84 GB (62.8% reduction)
-  No Missing Values
-  Duplicates Removed: 20
-  We have high imbalanced data (99.89% 0, 0.11% 1)
-  Saved to Bronze layer (Parquet format)

Flow: Load - Clean - Optimize - Save

**Next: - EDA**


